# Install all dependencies required

In [ ]:
#install pytorch
!pip install pyyaml==5.1

#install detectron2
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu102/torch1.9/index.html

#install easyocr and imutils
!pip install easyocr
!pip install imutils

In [ ]:
#checking pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
assert torch.__version__.startswith("1.9")

1.9.0+cu102 True


In [ ]:
#Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
import imutils
import easyocr
import pandas as pd
import time

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog


# Importing trained model

In [ ]:
cfg = get_cfg()
cfg.merge_from_file(os.path.join("/content/drive/MyDrive/detectron2_model/config.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
cfg.MODEL.WEIGHTS =  os.path.join("/content/drive/MyDrive/detectron2_model/model_final.pth")

# Testing on video

In [ ]:
#loading video
video = cv2.VideoCapture('/content/drive/MyDrive/testvideo_1.mp4')

#creating empty list for storing speed, numberplate and time
speed = []
char=[]
timing = []

#specifying characters to be removed from the recognised number plate characters
remov_chars = [';', ':', '!', '*', '@', '#', '$', '%', '^', '&', '(', ')', '[', ']', '?', '{','}']

#specifying coordinates for speed detection
coord=[[691,623],[1417,627],[439,703],[1527,705]]

#specifying the distance
dist = 11.8872



# is video opened or not
if (video.isOpened() == False): 
    print("Error reading video file")
  
#set resolution
frame_width = int(video.get(3))
frame_height = int(video.get(4))
   
size = (frame_width, frame_height)
   
#write video and output stores in outputvideo file
result = cv2.VideoWriter('outputvideo.avi', 
                         cv2.VideoWriter_fourcc(*'MJPG'),
                         10, size)
    
while(True):
    ret, frame = video.read()
  
    if ret == True: 
        
        #predicting licenseplate
        predictor = DefaultPredictor(cfg)
        outputs = predictor(frame)

        #boundingbox coordinates as array
        array = outputs["instances"].pred_boxes.tensor.cpu().numpy()
        prob = outputs["instances"].scores.cpu().numpy()
        
        if (array.size != 0):
            scr = str(prob[0])
            xmin, ymin, xmax, ymax = array[0][0], array[0][1], array[0][2], array[0][3]
            start_pt = (int(xmin)-5, int(ymin)-5)
            end_pt = (int(xmax)+5, int(ymax)+5)
            
            #drawing bounding box
            cv2.rectangle(frame, start_pt, end_pt,(255,255,255), 3)
            cv2.putText(frame, scr, end_pt, cv2.FONT_HERSHEY_SIMPLEX, 
                          1, (48, 217, 208), 2, cv2.LINE_AA)
            #drawing lines for speed detection
            cv2.line(frame, (coord[0][0],coord[0][1]),(coord[1][0],coord[1][1]),(0,0,255),2)   #First horizontal line
            cv2.line(frame, (coord[0][0],coord[0][1]), (coord[2][0],coord[2][1]), (0, 0, 255), 2) #Vertical left line
            cv2.line(frame, (coord[2][0],coord[2][1]), (coord[3][0], coord[3][1]), (0, 0, 255), 2) #Second horizontal line
            cv2.line(frame, (coord[1][0],coord[1][1]), (coord[3][0], coord[3][1]), (0, 0, 255), 2) #Vertical right line
             
            for (xmin, ymin, xmax, ymax) in array:

                # if coordinates of licenseplate pass first line, colour change to green and time will be noted as tim1
                if ( xmin >= coord[0][0] and ymin >= coord[0][1]):
                    cv2.line(frame, (coord[0][0], coord[0][1]), (coord[1][0], coord[1][1]), (0, 255,0), 2) #Changes line color to green
                    tim1 = time.time() #Initial time
                    converted_tim1 = str(tim1)
                    print(tim1)
                    cv2.putText(frame, 'Time : '+ converted_tim1, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 
                          2, (0, 0, 255), 2, cv2.LINE_AA) 
                    print("Car Entered.")

                 # if coordinates of licenseplate pass second line, colour change to green and time will be noted as tim2
                if ( xmin >= coord[2][0] and ymin >= coord[2][1]):
                    cv2.line(frame, (coord[2][0],coord[2][1]), (coord[3][0], coord[3][1]), (0, 0, 255), 2) #Changes line color to green
                    tim2 = time.time() #Final time
                    print(tim2)
                    print("Car Left.")
            
                   
                    # speed is calculated
                    spd_m = dist/(tim2-tim1) # speed in metre

                    spd_k = (spd_m * 3.6)   # speed in kilometre

                    spd = round(spd_k,2)    # rounding off to two decimal places
                    converted_spd = str(spd)
                    print("speed detected")
                    
                    
                    
            
            #cropped image of license plate
            cropped_img = frame[int(ymin)-5:int(ymax)+5, int(xmin)-5:int(xmax)+5]
        
             # Checking if the image is empty or not
            if cropped_img is not None:
                gray = cv2.cvtColor(cropped_img, cv2.COLOR_BGR2RGB)

                #reading text using easyocr
                reader = easyocr.Reader(['en'])
                res = reader.readtext(gray)
                
                #extracting only the characters
                if (res != []) :
                    x = res[0][1]

                    #removing undesired characters
                    for i in remov_chars :
                        x = x.replace(i, '')

                    #specifying length of characters
                    if  ((len(x.strip("\n")) >= 6) and (len(x.strip("\n")) <= 11)) :
                        
                        plate = x
                        #storing characters in a list
                        
                        cv2.putText(frame, 'Number plate : '+ plate, (50, 300), cv2.FONT_HERSHEY_SIMPLEX, 
                          2, (255, 255, 255), 3, cv2.LINE_AA)
                        cv2.putText(frame, 'Speed : '+ converted_spd + 'kmph', (50,200), cv2.FONT_HERSHEY_SIMPLEX, 
                          2, (255,0,0), 3, cv2.LINE_AA)
                        try:
                              #storing speed and time in lists
                            speed.append(spd)
                            timing.append(tim1)
                            char.append(plate)
                        except NameError:
                            print("Variable is not defined....!")

                            
                          
  

        # Write the frame into the
        # file 'output.avi'
        result.write(frame)
  
        # Display the frame
        # saved in the file
        cv2_imshow( frame)
  
    # Break the loop
    else:
        break
  
 
# write objects
video.release()
result.release()
    

   
print("The video was successfully saved")




In [ ]:
# Download the results
from google.colab import files
files.download('/content/outputvideo.avi')

In [ ]:
# generating output csv file
dict = {'Number plate': char, 'Speed': speed, 'Time': timing} 
df = pd.DataFrame(dict)
df.to_csv('output.csv')